Kalman Filter Functions as example

In [ ]:
#Kalman filter constants

    dt = 1.0
    u = np.array([[0.01],[0.01]])
    acc_noise = 0.1
    c_meas_noise = 0.1
    theta_meas_noise = 0.1
    Ez = np.array(   [[c_meas_noise,0],[0,theta_meas_noise]] )#measurement prediction error
    Ex = np.array( [[(dt**4)/4,0,(dt**3)/2,0],[0,(dt**4)/4,0,(dt**3)/2],[(dt**3)/2,0,(dt**2),0],[0,(dt**3)/2,0,(dt**2)]])* (acc_noise**2)#State prediction error
    #state and measurement equations
    A = np.array([[1,0,dt,0],[0,1,0,dt],[0,0,1,0],[0,0,0,1]] )
    B = np.array([[(dt**2)/2,0],[0,(dt**2)/2],[dt,0],[0,dt]] )
    C = np.array([[1,0,0,0],[0,1,0,0]] )


In [ ]:
#function to determine the state and covariance estimate of the system using Kalman's algorithm

def Kalman_filter_estimate(A,B,u,Ex,Q,P):
	'''
	Kalman_filter_estimate(A,B,u,Ex,Q,P) function is used to determint the state and the state covariance matrix of the described system using Kalman's algorithm

	input:  A - numpy.ndarray, State transition matrix of the system
			B - numpy.ndarray, Control correction matrix of the system
			u - numpy.ndarray, Control input to the system
			Ex - numpy.ndarray, State prediction error
			Q - numpy.ndarray, state of the system at the preceding time sample
			P - numpy.ndarray, state covariance matrix at the preceding time sample

	output: Q_est - numpy.ndarray, Estimated state at current time sample
		P_est - numpy.ndarray, Estimated state covariance matrix at current time sample 
	'''
	#Prediction part of Kalman Filter

	#Step 1: Est_of_x_at_t = (A * x_at_t-1) + (B * u_at_t-1)

	Q_est = dot_prod(A,Q) +  dot_prod(B,u) #prediction of state


	#Step 2: Est_of_state_cov_at_t = (A * state_cov_at_t-1 * A_transpose) + Ex

	P_est = dot_prod(dot_prod(A,P),A.transpose())+Ex #prediction of state covariance
	
	return (Q_est,P_est)


In [ ]:

#function to determine the updated state and state covariance matrices of the system using Kalman's algorithm
def Kalman_Filter_Update(C,Ex,Ez,Q_est, P_est, z ):
	'''
	Kalman_Filter_Update(C,Ex,Ez,Q_est, P_est, z ) function is used to calculate the updated state and state covariance matix through feedback z using Kalman's algorithm

	input:  C - numpy.ndarray, Measurement conversion matrix of the system
		Ex - numpy.ndarray, State prediction error
		Ez - numpy.ndarray, Measurement error
		Q_est - numpy.ndarray, Estimated state for the current time sample
		P_est - numpy.ndarray, Estimated state covariance matrix for the current time sample
		z - numpy.ndarray, Measurement matrix

	output: Q - numpy.ndarray, Updated state of the system for the current time sample
		P - numpy.ndarray, Updated state covariance matrix for the crrent time step
	'''
	if Q_est.size:

		#Update part of Kalman filter

		#Step 3: Kalman_gain_at_t = Est_of_state_cov_at_t * C_transpose ( C * Est_of_state_cov_at_t * C_transpose   +  Ez  )

		K = dot_prod( dot_prod(P_est,C.transpose()), np.linalg.inv(dot_prod(C,dot_prod(P_est,C.transpose()))+Ez)  )# Kalman gain (The weightage that needs to be given for the discrepency in measurement and the prediction to update the state and its covariance )

		#Step 4: State_at_t = Est_of_state_at_t + Kalman_gain_at_t (measured_variable_at_t - (C * Est_of_state_at_t) )

		Q = Q_est + dot_prod(K , (z - dot_prod(C,Q_est)  )  ) # correcting state estimate using measured variables to obtain actual state

		#Step 5: State_cov_at_t = (I - Kalman_gain_at_t * C) Est_of_state_cov_at_t

		P = dot_prod( (np.identity(4) - dot_prod(K,C)  ), P_est)# correcting state covariance estimate using measured variables to obtain actual state covariance 

	else:
		Q = np.array([[z[0][0]],[z[1][0]],[0],[0]])#state variable [[c],[theta],[c'],[theta']]
		P = Ex #Estimate of initial state covriance matrix
	return (Q,P)



In [ ]:


#Kalman Filter
if left_lane:

	(left_Q_est,left_P_est) = Kalman_filter_estimate(A,B,u,Ex,left_Q,left_P)
		
if right_lane:

    (right_Q_est,right_P_est) = Kalman_filter_estimate(A,B,u,Ex,right_Q,right_P)		



In [ ]:


#Elimination of Background based on Kalman Filtering
		
if left_Q_est.size and trial ==1:
	eft_img_pr = elimnate_predicted_background("left", left_img_pr, left_lane_img.shape[1],left_lane_img.shape[0], top_margin, left_lane_left_margin,  height, left_Q_est, prediction_error_tolerance )


if right_Q_est.size and trial ==1:

	right_img_pr = elimnate_predicted_background("right", right_img_pr, right_lane_img.shape[1], right_lane_img.shape[0], top_margin, right_lane_left_margin,  height, right_Q_est, prediction_error_tolerance )



In [ ]:

#measured intercept and angle values are used to update Kalman filter
if left_lane:

	(left_Q,left_P) = Kalman_Filter_Update(C,Ex,Ez,left_Q_est, left_P_est, np.array([[left_lane[-1]],[left_lane[4]]]) )

if right_lane:

	(right_Q,right_P) = Kalman_Filter_Update(C,Ex,Ez,right_Q_est, right_P_est, np.array([[right_lane[-1]],[right_lane[4]]]) )			


